In [35]:
import pandas as pd
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime
import time

In [118]:
#filtered records for UK visit, with duplicates
df_all = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\all_visitors.csv')
# df_all.shape
print("DataFrame before removing duplicates:\n", df_all.shape)
# # Convert 'datetime' to datetime type for accuracy
# df_all['datetime'] = pd.to_datetime(df_all['datetime'])
# df_all= df_all.sort_values(by=['maid','datetime','latitude','longitude'])
# # Remove duplicates and keep the first occurrence
df_unique = df_all.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'],keep='first').reset_index(drop=True)

# Display the DataFrame with duplicates removed
print("DataFrame after removing duplicates:\n", df_unique.shape)

DataFrame before removing duplicates:
 (19096604, 5)
DataFrame after removing duplicates:
 (8235248, 5)


In [119]:
df_unique.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\all_viistors_without_duplicate_without_rounding_off_lat_lon.csv', index= False)

In [121]:
df_all['latitude']=df_all['latitude'].astype(float).round(4)
df_all['longitude'] = df_all['longitude'].astype(float).round(4)
df_unique_2 = df_all.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'],keep='first').reset_index(drop=True)

# Display the DataFrame with duplicates removed
print("DataFrame after removing duplicates:\n", df_unique_2.shape)

DataFrame after removing duplicates:
 (6323096, 5)


In [122]:
df_unique_2.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\all_viistors_with_round_off_lat_lon.csv', index= False)

In [113]:
# df_all.drop(columns={'Unnamed: 0'}, inplace=True)
df_all.columns

Index(['maid', 'latitude', 'longitude', 'datetime'], dtype='object')

In [4]:
#stage 0
# original data
data_dist = pd.read_csv(r"C:\Users\User\Downloads\Mobility_traces_new_data_code\all visitors\final_filtered_data_with_state_and_district.csv")
data_dist['datetime']= pd.to_datetime(data_dist['datetime'])
data_dist.shape
data_dist= data_dist.sort_values(by=['maid','datetime','latitude','longitude'])

data_dist.rename(columns={'Instantaneous_speed(km/h)':'Velocity', 'consecutive_distance(kms)':'displacement', 'cumulative_distance(kms)':'distance'}, inplace=True)
data_dist.drop(columns={'consecutive_time_gap(minutes)','cumulative_time(minutes)'}, inplace=True)
data_dist.columns  
# Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
#        'Velocity', 'state', 'district'],
#       dtype='object')
data_dist.shape #(3726391, 9)

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district'],
      dtype='object')

In [42]:
# stage 1
# data_dist_without_type_1 = pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\whole data type 2 issue\rec with type 2 issue\original data without type 1\data_dist_without_type_1.csv')
data_dist_without_type_1.shape # (3593300, 14)
# data_dist_without_type_.drop(columns={'cumulative_time(minutes)'}, inplace=True)
data_dist_without_type_1.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'dist_previous_next',
       'time_gap_previous_next', 'next_valid_index', 'keep'],
      dtype='object')

In [14]:
# stage 3

# data_dist_without_type_2= pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\whole data type 2 issue\data_dist_without_type_2_with_airports.csv')
data_dist_without_type_2.shape # (3577471, 13)
# data_dist_without_type_2.drop(columns={'cumulative_time(minutes)'}, inplace=True)
data_dist_without_type_2.columns
# Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
#        'Velocity', 'state', 'district', 'Nearest airport',
#        'Distance to airport (km)', 'Distance Range', 'Coordinates of airport'],
#       dtype='object')

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'Nearest airport',
       'Distance to airport (km)', 'Distance Range', 'Coordinates of airport'],
      dtype='object')

In [15]:
# data_dist.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\data_dist.csv', index=False)
# data_dist_without_type_1.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\data_dist_without_type_1.csv', index=False)
# data_dist_without_type_2.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\data_dist_without_type_1_with_airport.csv', index=False)

In [17]:
# rec_with_high_speed_high_disp = data_dist_without_type_2[(data_dist_without_type_2['Velocity']>=150) &(data_dist_without_type_2['displacement']>=150)]
rec_with_high_speed_high_disp['maid'].nunique()

47357

In [27]:
# Group by 'maid' and count records per maid
record_counts_per_maid = rec_with_high_speed_high_disp.groupby('maid').size()

# Calculate statistics
max_count = record_counts_per_maid.max()
min_count = record_counts_per_maid.min()
mean_count = record_counts_per_maid.mean()
std_deviation = record_counts_per_maid.std()
# Count the number of 'maid' IDs with record count greater than X
count_ids_greater_than_X = (record_counts_per_maid > 5).sum()
# Print the results
print(f"Max count of records per maid: {max_count}")
print(f"Min count of records per maid: {min_count}")
print(f"Mean count of records per maid: {mean_count:.2f}")
print(f"Standard deviation of counts per maid: {std_deviation:.2f}")
print(f"Count of 'maid' IDs with record count greater than {5}: {count_ids_greater_than_X}")

Max count of records per maid: 13
Min count of records per maid: 1
Mean count of records per maid: 1.78
Standard deviation of counts per maid: 1.17
Count of 'maid' IDs with record count greater than 5: 743


In [20]:
rec_vel_150_disp_50 = data_dist_without_type_2[(data_dist_without_type_2['Velocity']>=150) &(data_dist_without_type_2['displacement']>=50)]
rec_vel_150_disp_50['maid'].count()

np.int64(105012)

In [28]:
# Group by 'maid' and count records per maid
record_counts_per_maid = rec_vel_150_disp_50.groupby('maid').size()

# Calculate statistics
max_count = record_counts_per_maid.max()
min_count = record_counts_per_maid.min()
mean_count = record_counts_per_maid.mean()
std_deviation = record_counts_per_maid.std()
# Count the number of 'maid' IDs with record count greater than X
count_ids_greater_than_X = (record_counts_per_maid > 5).sum()
# Print the results
print(f"Max count of records per maid: {max_count}")
print(f"Min count of records per maid: {min_count}")
print(f"Mean count of records per maid: {mean_count:.2f}")
print(f"Standard deviation of counts per maid: {std_deviation:.2f}")
print(f"Count of 'maid' IDs with record count greater than {5}: {count_ids_greater_than_X}")

Max count of records per maid: 14
Min count of records per maid: 1
Mean count of records per maid: 1.89
Standard deviation of counts per maid: 1.26
Count of 'maid' IDs with record count greater than 5: 1109


In [23]:
rec_vel_100_disp_50 = data_dist_without_type_2[(data_dist_without_type_2['Velocity']>=100) &(data_dist_without_type_2['displacement']>=50)]
rec_vel_100_disp_50['maid'].nunique() # 136154

62253

In [29]:
# Group by 'maid' and count records per maid
record_counts_per_maid = rec_vel_100_disp_50.groupby('maid').size()

# Calculate statistics
max_count = record_counts_per_maid.max()
min_count = record_counts_per_maid.min()
mean_count = record_counts_per_maid.mean()
std_deviation = record_counts_per_maid.std()
# Count the number of 'maid' IDs with record count greater than X
count_ids_greater_than_X = (record_counts_per_maid > 5).sum()
# Print the results
print(f"Max count of records per maid: {max_count}")
print(f"Min count of records per maid: {min_count}")
print(f"Mean count of records per maid: {mean_count:.2f}")
print(f"Standard deviation of counts per maid: {std_deviation:.2f}")
print(f"Count of 'maid' IDs with record count greater than {5}: {count_ids_greater_than_X}")

Max count of records per maid: 16
Min count of records per maid: 1
Mean count of records per maid: 2.19
Standard deviation of counts per maid: 1.56
Count of 'maid' IDs with record count greater than 5: 2738


In [57]:
rec_vel_150_disp_lt_50 = data_dist_without_type_2[(data_dist_without_type_2['Velocity']>=150) &(data_dist_without_type_2['displacement']<=50)]
rec_vel_150_disp_lt_50['maid'].count()

np.int64(191664)

In [58]:
# Group by 'maid' and count records per maid
record_counts_per_maid = rec_vel_150_disp_lt_50.groupby('maid').size()

# Calculate statistics
max_count = record_counts_per_maid.max()
min_count = record_counts_per_maid.min()
mean_count = record_counts_per_maid.mean()
std_deviation = record_counts_per_maid.std()
# Count the number of 'maid' IDs with record count greater than X
count_ids_greater_than_X = (record_counts_per_maid > 5).sum()
# Print the results
print(f"Max count of records per maid: {max_count}")
print(f"Min count of records per maid: {min_count}")
print(f"Mean count of records per maid: {mean_count:.2f}")
print(f"Standard deviation of counts per maid: {std_deviation:.2f}")
print(f"Count of 'maid' IDs with record count greater than {5}: {count_ids_greater_than_X}")

Max count of records per maid: 149
Min count of records per maid: 1
Mean count of records per maid: 3.72
Standard deviation of counts per maid: 3.90
Count of 'maid' IDs with record count greater than 5: 10135


IDs with vel>=150 and disp>=150

In [31]:
with open(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\whole data type 2 issue\rec_with_type_2_filtered\ids_with_vel_150_disp_150.json', 'r')  as f:
    ids_with_vel_150_disp_150 = json.load(f)

In [39]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Function to plot cumulative distance
def plot_cumulative_distance(df, prefix='', save_path=None):
    """
    Plots the cumulative distance against cumulative time for each maid in the dataframe and saves the figure to a specified path.

    Parameters:
    df (pd.DataFrame): DataFrame containing maid, datetime, cumulative_distance(kms).
    prefix (str): Prefix for the title of the plot.
    save_path (str): Path to save the plot image. If None, the plot will not be saved.
    """
    # Check and convert the datetime column if necessary
    if not pd.api.types.is_datetime64_any_dtype(df['datetime']):
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Plotting cumulative distance against cumulative time
    plt.figure(figsize=(12, 6))

    for maid, group in df.groupby('maid'):
        plt.plot(group['datetime'], group['distance'], marker='o', linestyle='-', label=maid)

    plt.xlabel('Date')
    plt.ylabel('Distance (Kms)')

    # Adjusting the legend position and appearance
    plt.legend(
        loc='upper left',
        title='Maid ID',
        fancybox=True,
        framealpha=0.5,
        borderpad=1
    )

    plt.title(f'{prefix} Distance vs Time')

    # Set the x-axis major locator to day and format the dates
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
    # ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))

    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()

    # Save the figure if save_path is provided
    if save_path:
        plt.savefig(save_path, format='png')  # Save as PNG file (you can change format if needed)
        print(f"Figure saved to {save_path}")

    plt.close()  # Close the figure to free memory


# Plot the cumulative distance vs time using the sample data
# plot_cumulative_distance(df)
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime

def create_position_time(df, shapefile_path, save_path):
    # Load shapefile using GeoPandas
    gdf = gpd.read_file(shapefile_path)

    # Convert datetime to pandas datetime
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Calculate time duration from a specific date (e.g., 2022-09-03)
    start_date = datetime(2022, 9, 3)
    df['time_duration'] = (df['datetime'] - start_date).dt.total_seconds() / 3600  # in hours
    max_duration = df['time_duration'].max()

    # Normalize time_duration to [0, 1] for heatmap intensity
    df['heatmap_intensity'] = df['time_duration'] / max_duration

    # Calculate bounding box based on dataframe coordinates
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

    # Calculate center of the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2

    # Create the map centered around the bounding box
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Uncomment if you want to add the GeoJSON layer
    # folium.GeoJson(gdf.to_json(), name="geojson").add_to(m)

    # Fit the map to the bounding box
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

    # Define VIBGYOR colors for markers
    vibgyor_colors = ['violet', 'indigo', 'blue', 'green', 'yellow', 'orange', 'red']

    # Define gradient colors for lines
    gradient_colors = ['green', 'lightgreen', 'yellow', 'orange', 'red', 'maroon']

    def get_gradient_color(speed):
        if speed > 600:
            return 'maroon'
        elif speed > 500:
            return 'red'
        elif speed > 400:
            return 'orange'
        elif speed > 300:
            return 'yellow'
        elif speed > 200:
            return 'lightgreen'
        else:
            return 'green'

    # Add lines connecting consecutive points with gradient colors
    for i in range(len(df) - 1):
        point1 = (df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        point2 = (df.iloc[i + 1]['latitude'], df.iloc[i + 1]['longitude'])
        
        # Determine color index based on heatmap intensity for markers
        intensity_index = int(df.iloc[i + 1]['heatmap_intensity'] * (len(vibgyor_colors) - 1))
        marker_color = vibgyor_colors[intensity_index]

        # Determine color for lines based on instantaneous speed
        speed = df.iloc[i + 1]['Velocity']
        line_color = get_gradient_color(speed)
        # Add markers with tooltips for each point
        tooltip1 = f"Maid: {df.iloc[i]['maid']}<br>Lat: {df.iloc[i]['latitude']}<br>Lon: {df.iloc[i]['longitude']}<br>Date: {df.iloc[i]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Displacement: {df.iloc[i]['displacement']}<br>Total Distance: {df.iloc[i]['distance']}<br>Inst. Speed(km/h): {df.iloc[i]['Velocity']}"
        folium.CircleMarker(
            location=point1,
            radius=5,
            color=marker_color,
            fill=True,
            fill_color=marker_color,
            fill_opacity=1,
            tooltip=tooltip1
        ).add_to(m)
        
        tooltip2 = f"Maid: {df.iloc[i+1]['maid']}<br>Lat: {df.iloc[i+1]['latitude']}<br>Lon: {df.iloc[i+1]['longitude']}<br>Date: {df.iloc[i+1]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Displacement: {df.iloc[i+1]['displacement']}<br>Total Distance: {df.iloc[i+1]['distance']}<br>Inst. Speed(km/h): {df.iloc[i+1]['Velocity']}"
        folium.CircleMarker(
            location=point2,
            radius=5,
            color=marker_color,
            fill=True,
            fill_color=marker_color,
            fill_opacity=1,
            tooltip=tooltip2
        ).add_to(m)

        # Add polyline connecting consecutive points with gradient color
        folium.PolyLine(locations=[point1, point2], color=line_color, weight=2.5, opacity=1).add_to(m)
    # Add starting and ending points with star markers
    start_point = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
    end_point = (df.iloc[-1]['latitude'], df.iloc[-1]['longitude'])
    folium.Marker(
        location=start_point,
        icon=folium.Icon(color='silver', icon='star'),
        tooltip='Start Point'
    ).add_to(m)
    folium.Marker(
        location=end_point,
        icon=folium.Icon(color='black', icon='star'),
        tooltip='End Point'
    ).add_to(m)
    # Save map to an HTML file
    m.save(save_path)

# Actual usage
# shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
# data_02a2 = data_dist[data_dist['maid']=='02a2c301-fb8c-4657-a346-a60fd50754d5']
# save_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\position_time_02a2c301-fb8c-4657-a346-a60fd50754d5.html'
# create_position_time(data_02a2, shapefile_path,save_path )



In [65]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm
#code to find the transition for displacements within a range from displacement_min to displacement_max
def calculate_distances_and_time_gaps_continous_invalid_stamps_with_disp_limitation(data, displacement_min, displacement_max, velocity_threshold, time_window, distance_range):
    # Convert 'datetime' column to datetime format
    data['datetime'] = pd.to_datetime(data['datetime'])
    # col_name = f'keep_for_{displacement_threshold}_{velocity_threshold}_{time_window}_{distance_range}'
    # Initialize new columns
    data['dist_previous_next'] = None
    data['time_gap_previous_next'] = None
    data['next_valid_index'] = None
    data['keep'] = True
    
    # Sort data by 'maid', 'datetime', 'latitude', and 'longitude', and reset index
    data = data.sort_values(by=['maid', 'datetime', 'latitude', 'longitude']).reset_index(drop=True)
    
    # Maintain a mapping from the original index to the new sorted index
    original_to_sorted_index = data.reset_index().set_index('index').index
    # Initialize the progress bar
    tqdm.pandas(desc="Processing maids")

    # Calculate distance and time gap
    for maid, group in tqdm(data.groupby('maid'), desc="Processing each maid"):
    # Calculate distance and time gap
    # for maid, group in data.groupby('maid'):
        num_records = len(group)
        for i in range(1, num_records - 1):
        # for i in range(1, num_records - 1):
            displacement = group.loc[group.index[i], 'displacement']
            # print(f'group.index[i]: {group.index[i]}')
            # print(f'disalcement:{displacement}')
            velocity = group.loc[group.index[i], 'Velocity']
            # print(f'disalcement:{displacement}, velocity { {velocity}}')
            if (displacement >= displacement_min) and (displacement<=displacement_max) and (velocity >= velocity_threshold):
                # print(f'Inside if: disalcement:{displacement}, velocity { {velocity}}')
                # print(f'index: {group.index[i]}')
                prev_record = group.loc[group.index[i - 1]]
                next_valid_index = None
                
                for j in range(i + 1, num_records):
                    next_record = group.loc[group.index[j]]
                    time_gap = (next_record['datetime'] - prev_record['datetime']).total_seconds() / 60
                    
                    if time_gap > time_window:
                        break
                    
                    distance = geodesic(
                        (prev_record['latitude'], prev_record['longitude']),
                        (next_record['latitude'], next_record['longitude'])
                    ).kilometers
                    
                    if distance < distance_range:
                        next_valid_index = j
                        break

                if next_valid_index is not None:
                    original_index = group.index[i]
                    sorted_next_valid_index = group.index[next_valid_index]

                    # Convert sorted index back to original index
                    original_next_valid_index = original_to_sorted_index.get_loc(sorted_next_valid_index)

                    # Update the current record (i) with information about the next valid index
                    data.at[original_index, 'next_valid_index'] = original_next_valid_index
                    data.at[original_index, 'dist_previous_next'] = distance
                    data.at[original_index, 'time_gap_previous_next'] = time_gap

                    # Mark records between the current index and one record before the next valid index as False
                    if original_next_valid_index > original_index:
                        indices_to_mark = range(original_index , original_next_valid_index)
                        data.loc[indices_to_mark, 'keep'] = False

    return data

import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

def calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward_with_disp_limit(data, displacement_min, displacement_max, velocity_threshold, time_window, distance_range):
    # Convert 'datetime' column to datetime format
    data['datetime'] = pd.to_datetime(data['datetime'])
    # col_name = f'keep_for_{displacement_threshold}_{velocity_threshold}_{time_window}_{distance_range}'
    # Initialize new columns
    data['dist_previous_next'] = None
    data['time_gap_previous_next'] = None
    data['prev_valid_index'] = None
    data['keep'] = True
    
    # Sort data by 'maid', 'datetime', 'latitude', and 'longitude', and reset index
    data = data.sort_values(by=['maid', 'datetime', 'latitude', 'longitude']).reset_index(drop=True)
    
    # Maintain a mapping from the original index to the new sorted index
    original_to_sorted_index = data.reset_index().set_index('index').index
    # Initialize the progress bar
    tqdm.pandas(desc="Processing maids")

    # Calculate distance and time gap
    for maid, group in tqdm(data.groupby('maid'), desc="Processing each maid"):
    # Calculate distance and time gap
    # for maid, group in data.groupby('maid'):
        num_records = len(group)
        for i in range(1, num_records - 1):
        # for i in range(1, num_records - 1):
            displacement = group.loc[group.index[i], 'displacement']
            # print(f'group.index[i]: {group.index[i]}')
            # print(f'disalcement:{displacement}')
            velocity = group.loc[group.index[i], 'Velocity']
            # print(f'disalcement:{displacement}, velocity { {velocity}}')
            if (displacement >= displacement_min) and (displacement<=displacement_max) and (velocity >= velocity_threshold):
                # print(f'Inside if: disalcement:{displacement}, velocity { {velocity}}')
                # print(f'index: {group.index[i]}')
                curr_record = group.loc[group.index[i]]
                prev_valid_index = None
                
                for j in range(i - 1, 0,-1):
                    prev_record = group.loc[group.index[j]]
                    # print(f'prev_record= {prev_record}')
                    # print(f'index of prev_record: {j}')
                    time_gap = (curr_record['datetime'] - prev_record['datetime']).total_seconds() / 60
                    # print(f'time_gap:{time_gap} {type(time_gap)}')
                    # print(f'time_window: {time_window} {type(time_window)}')
                    if time_gap > time_window:
                        
                        # print('break due to time gap')
                        break
                    
                    distance = geodesic(
                        (prev_record['latitude'], prev_record['longitude']),
                        (curr_record['latitude'], curr_record['longitude'])
                    ).kilometers
                    
                    if distance < distance_range:
                        prev_valid_index = j
                        # print('break due to distance')
                        break

                if prev_valid_index is not None:
                    original_index = group.index[i]
                    sorted_previous_valid_index = group.index[prev_valid_index]

                    # Convert sorted index back to original index
                    original_previous_valid_index = original_to_sorted_index.get_loc(sorted_previous_valid_index)
                    # print(f'original_index: {original_index}')
                    # print(f'original_previous_valid_index: {original_previous_valid_index}')
                    # Update the current record (i) with information about the next valid index
                    data.at[original_index, 'prev_valid_index'] = original_previous_valid_index
                    data.at[original_index, 'dist_previous_next'] = distance
                    data.at[original_index, 'time_gap_previous_next'] = time_gap

                    # Mark records between the one record previous to current index and one record later than the previous valid index as False
                    if original_previous_valid_index < original_index:
                        indices_to_mark = range(original_previous_valid_index + 1, original_index )
                        # print(f'indices_to_mark: {indices_to_mark}')
                        data.loc[indices_to_mark, 'keep'] = False

    return data

def filter_out_continuos_invalid_stamps(data):
    filtered_data = data[data['keep'] == True]
    return filtered_data



In [69]:
# data_dist_without_type_2 = data_dist_without_type_2.sort_values(by=['maid','datetime','latitude','longitude'])
# data_dist_with_info_of_smaller_displacements= calculate_distances_and_time_gaps_continous_invalid_stamps_with_disp_limitation(data_dist_without_type_2,25,50, 100,30,25 )
# data_dist_with_info_of_smaller_displacements = calculate_distances_and_time_gaps_continous_invalid_stamps_looking_backward_with_disp_limit(data_dist_without_type_2,10,25, 100,15,15 )

data_dist_with_info_of_smaller_displacements[data_dist_with_info_of_smaller_displacements['keep']==False]['maid'].count()

np.int64(695)

In [71]:
import numpy as np

In [93]:
x= data_dist[(data_dist['Velocity']>=150) & (data_dist['displacement']>=150)]
x['maid'].count()

np.int64(241246)

In [95]:
data_dist[(data_dist['Velocity']>=150)]['maid'].nunique()

85583

In [91]:
x= data_dist_without_type_2[(data_dist_without_type_2['Velocity']>=150) & (data_dist_without_type_2['displacement']>=150)]
x['maid'].count()

np.int64(84352)

In [101]:
import pandas as pd
import numpy as np

# # Sample DataFrame
# data = {
#     'maid': ['id1', 'id1', 'id1', 'id2', 'id2', 'id2', 'id2', 'id3', 'id3', 'id3', 'id3', 'id3', 'id3', 'id4', 'id4'],
#     'datetime': pd.to_datetime([
#         '2024-09-03', '2024-09-03', '2024-09-04', '2024-09-03', '2024-09-04', 
#         '2024-09-04', '2024-09-05', '2024-09-04', '2024-09-05', '2024-09-05',
#         '2024-09-06', '2024-09-06', '2024-09-06', '2024-09-07', '2024-09-07'
#     ]),
#     'lat': np.random.rand(15),
#     'long': np.random.rand(15)
# }

# df = pd.DataFrame(data)
df= data_dist.copy()

# Convert 'datetime' to date for daily grouping
df['date'] = df['datetime'].dt.date

# Group by 'maid' and 'date' to count timestamps for each maid per day
daily_maid_counts = df.groupby(['maid', 'date']).size().reset_index(name='count')

# Display the daily counts
# print("Daily counts per maid:\n", daily_maid_counts)

# Define the bins for daily counts
bins = [0,1,2,3, 5, 10, 20, 30, 50, 75, 100, 150, 200]

# Find the maximum number of records for any single maid on any given day to adjust the bins
max_count = daily_maid_counts['count'].max()

# Extend the bins to cover the maximum daily count dynamically
if max_count > 400:
    max_bin = ((max_count // 100) + 1) * 100
    bins.extend(range(500, max_bin + 100, 100))

# Categorize daily counts using pd.cut
daily_maid_counts['count_range'] = pd.cut(daily_maid_counts['count'], bins=bins, right=False)

# Count the number of unique maids in each range for each day
daily_range_counts = daily_maid_counts.groupby(['date', 'count_range'])['maid'].nunique().reset_index(name='unique_maid_count')

# Display the result
print("\nCount of IDs in each range of total timestamps per day:")
print(daily_range_counts)

# Optional: Display result in a more readable format for each day
for date in daily_range_counts['date'].unique():
    print(f"\nDate: {date}")
    daily_data = daily_range_counts[daily_range_counts['date'] == date]
    for index, row in daily_data.iterrows():
        print(f"Range {row['count_range']}: {row['unique_maid_count']} ID(s)")



Count of IDs in each range of total timestamps per day:
           date count_range  unique_maid_count
0    2022-09-03      [0, 1)                  0
1    2022-09-03      [1, 2)              17875
2    2022-09-03      [2, 3)              10691
3    2022-09-03      [3, 5)               8567
4    2022-09-03     [5, 10)               5709
..          ...         ...                ...
199  2022-09-14  [200, 500)                  2
200  2022-09-14  [500, 600)                  0
201  2022-09-14  [600, 700)                  1
202  2022-09-14  [700, 800)                  0
203  2022-09-14  [800, 900)                  1

[204 rows x 3 columns]

Date: 2022-09-03
Range [0, 1): 0 ID(s)
Range [1, 2): 17875 ID(s)
Range [2, 3): 10691 ID(s)
Range [3, 5): 8567 ID(s)
Range [5, 10): 5709 ID(s)
Range [10, 20): 689 ID(s)
Range [20, 30): 9 ID(s)
Range [30, 50): 1 ID(s)
Range [50, 75): 2 ID(s)
Range [75, 100): 1 ID(s)
Range [100, 150): 1 ID(s)
Range [150, 200): 0 ID(s)
Range [200, 500): 0 ID(s)
Range [500,

In [53]:
start= 0
end=10
sample_ids = ids_with_vel_150_disp_150[start:end]


In [54]:
# plotting origianl position_time_plot
sample_record_orig = data_dist[data_dist['maid'].isin(set(sample_ids))]
sample_record_orig['datetime']= pd.to_datetime(sample_record_orig['datetime'])
sample_record_orig=sample_record_orig.sort_values(by=['maid','datetime'])
sample_record_orig.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\Orig_{start}_{end}.xlsx', index=False)
plot_cumulative_distance(sample_record_orig, prefix='Original', save_path=fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\plots\Orig_{start}_{end}.png')

grouped = sample_record_orig.groupby('maid')
for maid, group in grouped:
    group['datetime']= pd.to_datetime(group['datetime'])
    group=group.sort_values(by=['datetime'])
    group.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\CSVs\orig_{maid}.csv', index=False)
    group.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\excel\orig_{maid}.xlsx', index=False)
    shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
    
    save_path = fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\pos_time\pos_time_orig_{maid}.html'
    create_position_time(group, shapefile_path,save_path )

Figure saved to C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\orig\plots\Orig_30_40.png


In [55]:
# plotting origianl position_time_plot
sample_record_without_type_1 = data_dist_without_type_1[data_dist_without_type_1['maid'].isin(set(sample_ids))]
sample_record_without_type_1['datetime']= pd.to_datetime(sample_record_without_type_1['datetime'])
sample_record_without_type_1=sample_record_without_type_1.sort_values(by=['maid','datetime'])
sample_record_without_type_1.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\Orig_{start}_{end}.xlsx', index=False)
plot_cumulative_distance(sample_record_without_type_1, prefix='without_type_1', save_path=fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\plots\without_type_1_{start}_{end}.png')

grouped = sample_record_without_type_1.groupby('maid')
for maid, group in grouped:
    group['datetime']= pd.to_datetime(group['datetime'])
    group=group.sort_values(by=['datetime'])
    group.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\CSVs\without_type_1_{maid}.csv', index=False)
    group.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\excel\without_type_1_{maid}.xlsx', index=False)
    shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
    
    save_path = fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\pos_time\without_type_1_{maid}.html'
    create_position_time(group, shapefile_path,save_path )

Figure saved to C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_1\plots\without_type_1_30_40.png


In [56]:
# plotting origianl position_time_plot
sample_record_without_type_2 = data_dist_without_type_2[data_dist_without_type_2['maid'].isin(set(sample_ids))]
sample_record_without_type_2['datetime']= pd.to_datetime(sample_record_without_type_2['datetime'])
sample_record_without_type_2=sample_record_without_type_2.sort_values(by=['maid','datetime'])
sample_record_without_type_2.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\Orig_{start}_{end}.xlsx', index=False)
plot_cumulative_distance(sample_record_without_type_2, prefix='without_type_2', save_path=fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\plots\without_type_2_{start}_{end}.png')

grouped = sample_record_without_type_2.groupby('maid')
for maid, group in grouped:
    group['datetime']= pd.to_datetime(group['datetime'])
    group=group.sort_values(by=['datetime'])
    group.to_csv(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\CSVs\without_type_2_{maid}.csv', index=False)
    group.to_excel(fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\excel\without_type_2_{maid}.xlsx', index=False)
    shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
    
    save_path = fr'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\pos_time\without_type_2_{maid}.html'
    create_position_time(group, shapefile_path,save_path )


Figure saved to C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\Comparing stages of filtering\without_type_2\plots\without_type_2_30_40.png


In [ ]:
start= 0
end=40
sample_ids = ids_with_vel_150_disp_150[start:end]


In [127]:
# df_unique = df_unique.sort_values(by=['maid','datetime','latitude','longitude'])
# df_unique_2 = df_unique_2.sort_values(by=['maid','datetime','latitude','longitude'])
df_unique= df_unique[['maid','datetime','latitude','longitude']]

df_unique_2= df_unique_2[['maid','datetime','latitude','longitude']]

In [128]:
sample_data_df_unique = df_unique[df_unique['maid'].isin(set(sample_ids))]
sample_data_df_unique_2 = df_unique_2[df_unique_2['maid'].isin(set(sample_ids))]
sample_data_without_type_1 = data_dist_without_type_1[data_dist_without_type_1['maid'].isin(set(sample_ids))]
sample_data_without_type_2 = data_dist_without_type_2[data_dist_without_type_2['maid'].isin(set(sample_ids))]


In [129]:
sample_data_without_type_2.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\comparing with raw data with unnecessary timestamps\sample_data_without_type_2.xlsx', index=False)
sample_data_without_type_1.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\comparing with raw data with unnecessary timestamps\sample_data_without_type_1.xlsx', index=False)
sample_data_df_unique_2.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\comparing with raw data with unnecessary timestamps\sample_data_with_round_off.xlsx', index=False)
sample_data_df_unique.to_excel(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\validating_records\comparing with raw data with unnecessary timestamps\sample_data_without_round_off.xlsx', index=False)

In [102]:
with open(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\all_useful_visitors_statewise_date_wise_counts\bkp\maid_with_NaN_values_of_states_new_geojson.json', 'r') as f:
    nan_ids = json.load(f)

In [140]:
sample_data_df_unique.index.level

AttributeError: 'Index' object has no attribute 'level'

In [103]:
len(nan_ids)

3